In [4]:
!pip install -qU langchain openai langchain-openai
!pip install -qU langchain-community

In [5]:
import json
import os
import pandas as pd
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_openai import ChatOpenAI

In [6]:
os.environ["OPENAI_API_KEY"] = "..."
try:
    from google.colab import userdata

    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    if not os.environ["OPENAI_API_KEY"]:
        raise ValueError("Colab의 보안 비밀 설정에서 'OPENAI_API_KEY'를 찾을 수 없습니다.")
except ImportError:
    print("Colab 환경이 아니므로, 'api_key' 변수에 직접 값을 할당해야 합니다.")

Colab 환경이 아니므로, 'api_key' 변수에 직접 값을 할당해야 합니다.


In [7]:
simple_question = "메탈리카(Metallica) 멤버는 누구인가요? 쉼표로 구분해 나열해 주세요. 멤버 이름 외 다른 텍스트는 생성하지 마세요."
chat = ChatOpenAI(model_name="gpt-4o-mini")
chat_prompt = PromptTemplate.from_template(simple_question)
chain = chat_prompt | chat
result = chain.invoke({})
print(result.content)

제임스 헷필드, 라스 울리히, 키스 해먼드, 로버트 트루지로, 클리프 버튼


In [8]:
request_list_format = """주기율표에서 처음 10개의 원소를 쉼표로 구분하여 작성해 주세요.
원소 외에 다른 텍스트는 생성하지 마세요."""

prompt = PromptTemplate(template=request_list_format, input_variables=[])
output_parser = CommaSeparatedListOutputParser()
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)
chain = prompt | chat | output_parser
result = chain.invoke({})
print(result)

['수소', '헬륨', '리튬', '베릴륨', '붕소', '탄소', '질소', '산소', '플루오르', '네온']


In [9]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 대화형 AI 비서야. 이전 대화를 반드시 참고해서 대답해."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])
session_histories = {}


def get_session_history(session_id):
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()
    return session_histories[session_id]


chain = RunnableWithMessageHistory(
    runnable=prompt | llm,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)
first_input = "당신이 알고 있는 대한민국의 공휴일 10가지를 쉼표로 구분하여 나열해 주세요."
first_result = chain.invoke(
    {"input": first_input},
    config={"configurable": {"session_id": "user-1"}}
)
print(f"첫 번째 응답:\n{first_result.content}\n")

첫 번째 응답:
설날, 추석, 삼일절, 어린이날, 광복절, 개천절, 한글날, 석가탄신일, 현충일, 크리스마스.



In [10]:
second_input = "방금 나열한 공휴일 목록에서 종교적인 공휴일을 제거해 주세요."
second_result = chain.invoke(
    {"input": second_input},
    config={"configurable": {"session_id": "user-1"}}
)
print(f"두 번째 응답:\n{second_result.content}\n")

두 번째 응답:
설날, 추석, 삼일절, 어린이날, 광복절, 개천절, 한글날, 현충일.



In [11]:
advanced_data_structure = """각 공휴일에 대해 두 문장으로 설명하세요. 출력은 JSON 형식의 표로 작성해 주세요.
표의 이름은 "holidays"이고, 필드는 "name"과 "description"입니다. 각 행에서 "name"은 공휴일의 이름이고,
"description"은 생성된 설명입니다. 출력의 구문은 줄 바꿈 없이 JSON 형식이어야 합니다.

예시:
{"holidays": [
        {"name": "holiday_name",
         "description": "holiday_description"
        }
        ]}
"""
output = chain.invoke(
    {"input": advanced_data_structure},
    config={"configurable": {"session_id": "user-1"}}
)
print(type(output))
response_text = output.content
parsed_json = json.loads(response_text)
holidays = parsed_json.get("holidays", [])
print(holidays)

<class 'langchain_core.messages.ai.AIMessage'>
[{'name': '설날', 'description': '설날은 음력 1월 1일로, 한국의 전통적인 새해입니다. 가족들이 모여 차례를 지내고 덕담을 나누는 날입니다.'}, {'name': '추석', 'description': '추석은 음력 8월 15일로, 한국의 대표적인 명절 중 하나입니다. 가족들이 모여 성묘를 하고 송편을 나눠 먹습니다.'}, {'name': '삼일절', 'description': '삼일절은 3월 1일로, 1919년 독립운동을 기념하는 날입니다. 전국적으로 기념식과 행사가 열립니다.'}, {'name': '어린이날', 'description': '어린이날은 5월 5일로, 어린이들의 행복과 권리를 기념하는 날입니다. 부모와 아이들이 함께 즐거운 시간을 보냅니다.'}, {'name': '광복절', 'description': '광복절은 8월 15일로, 1945년 일본으로부터의 해방을 기념하는 날입니다. 다양한 기념식과 행사가 열립니다.'}, {'name': '개천절', 'description': '개천절은 10월 3일로, 단군이 고조선을 건국한 것을 기념하는 날입니다. 한국의 건국 신화를 기념합니다.'}, {'name': '한글날', 'description': '한글날은 10월 9일로, 세종대왕이 한글을 반포한 것을 기념하는 날입니다. 한글의 중요성과 우수성을 기리는 날입니다.'}, {'name': '현충일', 'description': '현충일은 6월 6일로, 전쟁에서 희생된 이들을 기리는 날입니다. 전국적으로 묵념과 추모 행사가 진행됩니다.'}]


In [12]:
pd.set_option('display.max_colwidth', None)
dict_content = json.loads(output.content)
pd.DataFrame(dict_content['holidays'])

,name,description
0,설날,"설날은 음력 1월 1일로, 한국의 전통적인 새해입니다. 가족들이 모여 차례를 지내고 덕담을 나누는 날입니다."
1,추석,"추석은 음력 8월 15일로, 한국의 대표적인 명절 중 하나입니다. 가족들이 모여 성묘를 하고 송편을 나눠 먹습니다."
2,삼일절,"삼일절은 3월 1일로, 1919년 독립운동을 기념하는 날입니다. 전국적으로 기념식과 행사가 열립니다."
3,어린이날,"어린이날은 5월 5일로, 어린이들의 행복과 권리를 기념하는 날입니다. 부모와 아이들이 함께 즐거운 시간을 보냅니다."
4,광복절,"광복절은 8월 15일로, 1945년 일본으로부터의 해방을 기념하는 날입니다. 다양한 기념식과 행사가 열립니다."
5,개천절,"개천절은 10월 3일로, 단군이 고조선을 건국한 것을 기념하는 날입니다. 한국의 건국 신화를 기념합니다."
6,한글날,"한글날은 10월 9일로, 세종대왕이 한글을 반포한 것을 기념하는 날입니다. 한글의 중요성과 우수성을 기리는 날입니다."
7,현충일,"현충일은 6월 6일로, 전쟁에서 희생된 이들을 기리는 날입니다. 전국적으로 묵념과 추모 행사가 진행됩니다."
